# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-12-27&until-occurred-date=2022-01-03


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

26919 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-12-27&until-occurred-date=2022-01-03
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=21ee8ccf-00e9-4109-816e-c91cf9848a98&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-12-27&until-occurred-date=2022-01-03
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=79859332-466d-4e53-bd66-48495e51ee0d&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-12-27&until-occurred-date=2022-01-03
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-10-18.csv
failed to load preprint_tweets_2021-10-25.csv
failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-12-20.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load preprint_tweets_2021-12-27.csv
failed to load preprint_tweets_2021-10-23.csv
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-11-01.csv
failed to load preprint_tweets_2021-11-15.csv
failed to load preprint_tweets_2021-11-29.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-11-08.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load preprint_tweets_2021-11-22.csv
failed to load preprint_tweets_2021-10-05.csv
failed to load preprint_tweets_2021-10-11.csv
failed to load preprint_tweets_2021-10-04.csv
failed to 

In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.12.20.21267966...
REST API query complete  200
REST API query started for 10.1101/2021.06.11.21258690...
REST API query complete  200
REST API query started for 10.1101/2021.12.24.474084...
REST API query complete  200
REST API query started for 10.1101/2021.10.11.21264863...
REST API query complete  200
REST API query started for 10.1101/2021.12.18.21268018...
REST API query complete  200
REST API query started for 10.1101/2021.08.10.21261846...
REST API query complete  200
REST API query started for 10.1101/2021.07.08.21260210...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.12.17.473248...
REST API query complete  200
REST API query started for 10.1101/2021.08.24.21262415...
REST API query complete  200
REST API query started for 10.1101/2020.12.24.20248802...
REST API query complete  200
REST API query started for 10.1101/2021.12.28.2

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.12.28.21268436,382,medRxiv,Infectious Diseases (except HIV/AIDS),True,Vaccine effectiveness against hospital admissi...,"[{'name': 'Glenda E Gray'}, {'name': 'Shirley ...",<p>Following the results of the ENSEMBLE 2 stu...,2021-12-29
1,10.1101/2021.12.25.474149,71,bioRxiv,Microbiology,True,Expanded ACE2 dependencies of diverse SARS-lik...,"[{'name': 'Sarah M Roelle'}, {'name': 'Nidhi S...",<p>Viral spillover from animal reservoirs can ...,2021-12-27
2,10.1101/2021.12.29.474455,62,bioRxiv,Developmental Biology,False,Nerve-mediated amputation-induced stem cell ac...,"[{'name': 'Duygu Payzin-Dogru'}, {'name': 'Sar...",<p>Animals exhibit extreme diversity in regene...,2021-12-30
3,10.1101/2021.12.29.474457,59,bioRxiv,Evolutionary Biology,False,Effects of periodic bottlenecks on the dynamic...,"[{'name': 'Minako Izutsu'}, {'name': 'Devin M....",<p>Population bottlenecks are common in nature...,2021-12-30
4,10.1101/2021.12.30.474555,55,bioRxiv,Bioengineering,False,A nanopore interface for high bandwidth DNA co...,"[{'name': 'Karen Zhang'}, {'name': 'Yuan-Jyue ...",<p>DNA has emerged as a powerful substrate for...,2021-12-30
5,10.1101/2021.12.29.474489,50,bioRxiv,Neuroscience,False,High density single-unit human cortical record...,"[{'name': 'Jason E Chung'}, {'name': 'Kristin ...",<p>A fundamental unit of neural computation is...,2021-12-30
6,10.1101/2021.12.28.474382,46,bioRxiv,Cell Biology,False,Precise measurement of nanoscopic septin ring ...,"[{'name': 'Amin Zehtabian'}, {'name': 'Paul Ma...",<p>The combination of image analysis and fluor...,2021-12-29
7,10.1101/2021.12.23.21268179,44,medRxiv,Epidemiology,False,An online atlas of human plasma metabolite sig...,"[{'name': 'Koen F. Dekkers'}, {'name': 'Sergi ...",<p>The human gut microbiota produces a variety...,2021-12-30
8,10.1101/2021.12.28.474304,40,bioRxiv,Neuroscience,False,Techniques for visualizing fibroblast-vessel i...,"[{'name': 'Hannah E Jones'}, {'name': 'Kelsey ...",<p>Fibroblasts are found associated with blood...,2021-12-28
9,10.1101/2021.12.29.474385,37,bioRxiv,Genomics,False,Single-nuclei isoform RNA sequencing reveals c...,"[{'name': 'Simon A Hardwick'}, {'name': 'Wen H...",<p>Single-nuclei RNA-Seq is being widely emplo...,2021-12-30


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

76 preprints (including 13 covering SARS-CoV-2, 62 from bioRxiv and 14 from medRxiv) published in the last 7 days before 2022-01-03 had been tweeted at least 3 times (maximum 382).